In [1208]:
import numpy as np

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x)) 

def relu(x):
    return x * (x > 0)

In [1209]:
class Neuron:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias
        
    def feedforward(self, inputs, act_type):
        total = np.dot(self.weight, inputs) + self.bias
        if act_type == 0:
            return sigmoid(total)
        elif act_type == 1:
            return softmax(total)
        elif act_type == 2:
            return relu(total)

In [1210]:
class Neural_Net:
    def __init__(self, h_num, o_num, i_weight, o_weight, i_bias, o_bias):
        self.i_weight = i_weight
        self.o_weight = o_weight
        self.i_bias = i_bias
        self.o_bias = o_bias
        self.h_num = h_num
        self.o_num = o_num
        self.hidden = []
        self.output = []
        for i in range(h_num):
            self.hidden.append(Neuron(i_weight[i,:], i_bias[i]))
        for i in range(o_num):
            self.output.append(Neuron(o_weight[i,:], o_bias[i]))
    def feedforward(self, i_data):
        o_hidden = []
        o_output = []
        for i in range(self.h_num):
            o_hidden.append(self.hidden[i].feedforward(i_data,0))
        for i in range(self.o_num):
            o_output.append(self.output[i].feedforward(o_hidden,0))
        return o_output, o_hidden
    def backpropagation(self, i_y, y_pred, h_value, i_x):
        h_field = np.matmul(np.matmul(deriv_loss(i_y,y_pred), deriv_pred(h_value,self.o_weight,self.o_bias)), deriv_hidden(i_x, self.i_weight, self.i_bias))
        o_field = np.matmul(np.transpose(deriv_loss(i_y,y_pred)), deriv_hidden(h_value, self.o_weight, self.o_bias))
        h_bias_field = h_field[len(h_field)-self.h_num:]
        h_field = np.reshape(h_field[0:len(h_field)-self.h_num], (self.h_num, len(i_x)))#h_field[0:len(h_field)-self.h_num].reshape[self.h_num, self.o_num]
        o_bias_field = o_field[len(o_field)-self.o_num:]
        o_field = np.reshape(o_field[0:len(o_field)-self.o_num], (self.o_num, self.h_num))
        return h_field, o_field, h_bias_field, o_bias_field

In [1211]:
def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def deriv_loss(y_true, y_pred):
    return -2*(y_true - y_pred)

def deriv_pred(hidden, o_weight, o_bias):
    return o_weight * deriv_sigmoid(np.dot(o_weight, hidden) + o_bias)[:,None]

def deriv_hidden(i_data, i_weight, i_bias):
    n_dim = i_weight.shape[0]
    w_dim = i_weight.shape[1]
    derv = np.zeros((n_dim, n_dim * w_dim + n_dim))
    for i in range(0,n_dim):
        derv[i : i+1, i*w_dim : (i+1)*w_dim] = np.array(i_data) * deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
        derv[i : i+1, n_dim*w_dim+i : n_dim*w_dim+i+1] = deriv_sigmoid(np.dot(i_weight[i,:],i_data) + i_bias[i])
    return derv

In [1212]:
i_weight = np.array([[0.3,0.7],[0.1,0.2],[0.4,0.1],[0.2,0.5]])
o_weight = np.array([[0.4,0.3,0.2,0.1]])
i_bias = np.array([0.5,0.4,0.3,0.5])
o_bias = np.array([1])
i_data = np.array([[1,0,1,0],[1,0,1,1]])
i_data = np.array([
  [-2, -1, 1],  # Alice
  [25, 6, 0],   # Bob
  [17, 4, 0],   # Charlie
  [-15, -6, 1], # Diana
])

x_i_weight = 2
y_i_weight = 2
i_weight = np.random.normal(size=(x_i_weight,y_i_weight))

x_o_weight = 1
y_o_weight = 2
o_weight = np.random.normal(size=(x_o_weight,y_o_weight))

x_i_bias = 1
y_i_bias = 2
i_bias = np.random.normal(size=(y_i_bias))

x_o_bias = 1
y_o_bias = 1
o_bias = np.random.normal(size=(x_o_bias))

print(i_weight)
print(o_weight)
print(i_bias)
print(o_bias)

[[-0.99979337  1.55506309]
 [-0.91283335 -0.84163001]]
[[-0.83033726  0.03988017]]
[1.79068143 0.81410694]
[0.322045]


In [1213]:
def train(i_weight, o_weight, i_bias, o_bias, i_data, learning_rate):
    epoch = 1000
    h_size = i_weight.shape[0]
    o_size = i_data.shape[1]
    n = i_data.shape[1]
    obs_n = i_data.shape[0]
    i_y = i_data[: , n-1:n]
    o_size = i_y.shape[1]
    i_x = i_data[: , 0:n-1]
    
    for j in range(epoch):
        loss = 0
        for i in range(0,obs_n):
            network = Neural_Net(h_size, o_size, i_weight, o_weight, i_bias, o_bias)
            pred_y, h_value = network.feedforward(i_x[i, :])
            h_field, o_field, h_bias_field, o_bias_field = network.backpropagation(i_y[i], pred_y, h_value, i_x[i, :])
            i_weight, o_weight, i_bias, o_bias = gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate)
            loss = loss + mse_loss(i_y[i], pred_y)
        print("Epoch %d loss: %.3f" % (epoch, loss))
    #return i_weight, o_weight, i_bias, o_bias
    
        
def gradient_descent(i_weight, o_weight, i_bias, o_bias, h_field, o_field, h_bias_field, o_bias_field, learning_rate):
    i_weight = i_weight - learning_rate * h_field
    o_weight = o_weight - learning_rate * o_field
    i_bias = i_bias - learning_rate * h_bias_field
    o_bias = o_bias - learning_rate * o_bias_field
    return i_weight, o_weight, i_bias, o_bias

def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [1214]:
train(i_weight, o_weight, i_bias, o_bias, i_data, 0.1)

Epoch 1000 loss: 1.407
Epoch 1000 loss: 1.345
Epoch 1000 loss: 1.286
Epoch 1000 loss: 1.232
Epoch 1000 loss: 1.181
Epoch 1000 loss: 1.134
Epoch 1000 loss: 1.091
Epoch 1000 loss: 1.052
Epoch 1000 loss: 1.015
Epoch 1000 loss: 0.981
Epoch 1000 loss: 0.949
Epoch 1000 loss: 0.920
Epoch 1000 loss: 0.892
Epoch 1000 loss: 0.865
Epoch 1000 loss: 0.840
Epoch 1000 loss: 0.816
Epoch 1000 loss: 0.793
Epoch 1000 loss: 0.771
Epoch 1000 loss: 0.750
Epoch 1000 loss: 0.730
Epoch 1000 loss: 0.710
Epoch 1000 loss: 0.691
Epoch 1000 loss: 0.673
Epoch 1000 loss: 0.655
Epoch 1000 loss: 0.638
Epoch 1000 loss: 0.622
Epoch 1000 loss: 0.605
Epoch 1000 loss: 0.590
Epoch 1000 loss: 0.575
Epoch 1000 loss: 0.560
Epoch 1000 loss: 0.546
Epoch 1000 loss: 0.533
Epoch 1000 loss: 0.519
Epoch 1000 loss: 0.507
Epoch 1000 loss: 0.494
Epoch 1000 loss: 0.482
Epoch 1000 loss: 0.471
Epoch 1000 loss: 0.460
Epoch 1000 loss: 0.449
Epoch 1000 loss: 0.438
Epoch 1000 loss: 0.428
Epoch 1000 loss: 0.418
Epoch 1000 loss: 0.409
Epoch 1000 

Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.028
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.027
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.026
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 loss: 0.025
Epoch 1000 

Epoch 1000 loss: 0.013
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 loss: 0.012
Epoch 1000 

In [1215]:
emily = np.array([-7, -3]) # 128 pounds, 63 inches
frank = np.array([20, 2])  # 155 pounds, 68 inches
p1,h1 = network.feedforward(emily)
p2,h2 = network.feedforward(frank)
print("Emily: %.3f" % p1[0]) # 0.951 - F
print("Frank: %.3f" % p2[0]) # 0.039

Emily: 0.736
Frank: 0.881
